In [11]:
import pandas as pd
from django.db.models import F, Value, CharField
from django.db.models.functions import Concat
from project.models import Project
from public_data.models import CommunePop, Cerema, CommuneDiff

project = Project.objects.all().first()

In [2]:
headers = [
    "Commune",
    "Insee",
    "EPCI",
    "SCoT",
    "Département",
    "Région",
    "Année",
    "Changement",
    "Total",
]
qs = (
    CommunePop.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        SCoT=F("city__scot__name"),
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Année=F("year"),
        Changement=F("pop_change"),
        Total=F("pop"),
    )
    .values(*headers)
)
qs[:10]

<QuerySet [{'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2019, 'Changement': -1, 'Total': 46}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2018, 'Changement': 0, 'Total': 47}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2017, 'Changement': -1, 'Total': 47}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2016, 'Changement': -1, 'Total': 48}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2015, 'Changement': -1, 'Total': 49}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI

In [3]:
df = (
    pd.DataFrame(qs, columns=headers)
    .fillna(0.0)
    .pivot(columns=["Année"], index=headers[:6], values=["Changement", "Total"])
)
df

Changement  \
Année                                                                                      2006   
Commune              Insee EPCI                           SCoT Département Région                 
Antras               32003 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Auch                 32013 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Augnax               32014 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Auterive             32019 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Ayguetinte           32024 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Biran                32054 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Bonas                32059 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castelnau-Barbarens  32076 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castillon-Massas     32089 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castin               32091 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castéra-Verduzan     32083 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Crastes              32112 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Duran                32117 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Jegun                32162 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Lahitte              32183 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Lavardens            32204 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Leboulin             32207 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Mirepoix             32258 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Montaut-les-Créneaux 32279 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Montégut             32282 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Mérens               32251 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Nougaroulet          32298 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Ordan-Larroque       32301 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Pavie                32307 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Pessan               32312 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Peyrusse-Massas      32316 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Preignan             32331 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Puycasquier          32335 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Roquefort            32347 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Roquelaure           32348 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Saint-Jean-Poutge    32382 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Saint-Lary           32384 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Sainte-Christie      32368 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Tourrenquets         32453 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   

                                                                                             \
Année                                                                                  2007   
Commune              Insee EPCI                           SCoT Département Région             
Antras               32003 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie    0.0   
Auch                 32013 CA Grand Auch 

In [4]:
headers = [
    "Commune",
    "Insee",
    "EPCI",
    "SCoT",
    "Département",
    "Région",
    "Année",
    "Changement",
    "Total",
]
qs = (
    CommunePop.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        SCoT=F("city__scot__name"),
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Année=F("year"),
        Changement=F("household_change"),
        Total=F("household"),
    )
    .values(*headers)
)
df = (
    pd.DataFrame(qs, columns=headers)
    .fillna(0.0)
    .pivot(columns=["Année"], index=headers[:6], values=["Changement", "Total"])
)
df

Changement  \
Année                                                                                      2006   
Commune              Insee EPCI                           SCoT Département Région                 
Antras               32003 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Auch                 32013 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Augnax               32014 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Auterive             32019 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Ayguetinte           32024 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Biran                32054 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Bonas                32059 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castelnau-Barbarens  32076 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castillon-Massas     32089 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castin               32091 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castéra-Verduzan     32083 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Crastes              32112 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Duran                32117 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Jegun                32162 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Lahitte              32183 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Lavardens            32204 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Leboulin             32207 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Mirepoix             32258 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Montaut-les-Créneaux 32279 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Montégut             32282 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Mérens               32251 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Nougaroulet          32298 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Ordan-Larroque       32301 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Pavie                32307 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Pessan               32312 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Peyrusse-Massas      32316 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Preignan             32331 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Puycasquier          32335 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Roquefort            32347 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Roquelaure           32348 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Saint-Jean-Poutge    32382 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Saint-Lary           32384 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Sainte-Christie      32368 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Tourrenquets         32453 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   

                                                                                           \
Année                                                                                2007   
Commune              Insee EPCI                           SCoT Département Région           
Antras               32003 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie  0.0   
Auch                 32013 CA Grand Auch Cœur de 

In [18]:
qs = (
    CommuneSol.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        # "SCoT": "city__scot__name",
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
    )
    .filter(matrix__is_artificial=True)
    .values("Commune", "Insee", "EPCI", "Département", "Région")
    .annotate(Surface_artificielle_dernier_millésime=Sum("surface"))
    .order_by("Commune", "Insee", "EPCI", "Département", "Région")
)
df = pd.DataFrame(
    qs,
    columns=[
        "Commune",
        "Insee",
        "EPCI",
        "Département",
        "Région",
        "Surface_artificielle_dernier_millésime",
    ],
).fillna(0.0)
df = df.set_index(["Commune",
        "Insee",
        "EPCI",
        "Département",
        "Région"])
df

,,,,,Surface_artificielle_dernier_millésime
Commune,Insee,EPCI,Département,Région,
Antras,32003,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,34.6000
Auch,32013,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,3380.9900
Augnax,32014,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,47.3500
Auterive,32019,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,184.8200
Ayguetinte,32024,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,78.9200
Biran,32054,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,254.8600
Bonas,32059,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,134.9300
Castelnau-Barbarens,32076,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,313.3200
Castillon-Massas,32089,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,109.9300


In [22]:
qs = (
    CommuneDiff.objects.filter(city__in=project.cities.all())
    .filter(year_old__gte=project.analyse_start_date)
    .filter(year_old__lte=project.analyse_end_date)
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        # "SCoT": "city__scot__name",
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Différentiel=Concat("year_old", Value("-"), "year_new", output_field=CharField()),
        Artificialisation=F("new_artif"),
        Renaturation=F("new_natural"),
        Artificialisation_nette=F("net_artif"),
    )
    .values("Commune",
            "Insee",
            "EPCI",
            "Département",
            "Région",
            "Différentiel",
            "Artificialisation",
            "Renaturation",
            "Artificialisation_nette")
)
qs

<QuerySet [{'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Différentiel': '2016-2019', 'Artificialisation': Decimal('0.0000'), 'Renaturation': Decimal('0.0100'), 'Artificialisation_nette': Decimal('-0.0100')}, {'Commune': 'Auch', 'Insee': '32013', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Différentiel': '2016-2019', 'Artificialisation': Decimal('25.7900'), 'Renaturation': Decimal('2.1500'), 'Artificialisation_nette': Decimal('23.6400')}, {'Commune': 'Augnax', 'Insee': '32014', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Différentiel': '2016-2019', 'Artificialisation': Decimal('1.2700'), 'Renaturation': Decimal('0.0600'), 'Artificialisation_nette': Decimal('1.2100')}, {'Commune': 'Auterive', 'Insee': '32019', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Différentiel': '2016

In [23]:
df2 = (
    pd.DataFrame(
        qs,
        columns=[
            "Commune",
            "Insee",
            "EPCI",
            "Département",
            "Région",
            "Différentiel",
            "Artificialisation",
            "Renaturation",
            "Artificialisation_nette",
        ],
    )
    .fillna(0.0)
    .pivot(
        columns=["Différentiel"],
        index=["Commune", "Insee", "EPCI", "Département", "Région"],
        values=[
            "Artificialisation",
            "Renaturation",
            "Artificialisation_nette",
        ],
    )
)
df2

,,,,,Artificialisation,Renaturation,Artificialisation_nette
,,,,Différentiel,2016-2019,2016-2019,2016-2019
Commune,Insee,EPCI,Département,Région,,,
Antras,32003,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,0.0000,0.0100,-0.0100
Auch,32013,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,25.7900,2.1500,23.6400
Augnax,32014,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,1.2700,0.0600,1.2100
Auterive,32019,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,1.2500,0.0000,1.2500
Ayguetinte,32024,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,0.0900,0.0000,0.0900
Biran,32054,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,1.2300,0.4100,0.8200
Bonas,32059,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,0.0300,0.0000,0.0300
Castelnau-Barbarens,32076,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,1.1500,0.0200,1.1300


In [27]:
df.merge(df2, left_index=True, right_index=True)

/tmp/ipykernel_62/2027072862.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df.merge(df2, left_index=True, right_index=True)


,,,,,Surface_artificielle_dernier_millésime,"(Artificialisation, 2016-2019)","(Renaturation, 2016-2019)","(Artificialisation_nette, 2016-2019)"
Commune,Insee,EPCI,Département,Région,,,,
Antras,32003,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,34.6000,0.0000,0.0100,-0.0100
Auch,32013,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,3380.9900,25.7900,2.1500,23.6400
Augnax,32014,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,47.3500,1.2700,0.0600,1.2100
Auterive,32019,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,184.8200,1.2500,0.0000,1.2500
Ayguetinte,32024,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,78.9200,0.0900,0.0000,0.0900
Biran,32054,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,254.8600,1.2300,0.4100,0.8200
Bonas,32059,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,134.9300,0.0300,0.0000,0.0300
Castelnau-Barbarens,32076,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,313.3200,1.1500,0.0200,1.1300
Castillon-Massas,32089,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,109.9300,0.0000,0.0000,0.0000


In [48]:
df3 = df.copy()
df3 = df3.astype(float)
df3

,,,,,Surface_artificielle_dernier_millésime
Commune,Insee,EPCI,Département,Région,
Antras,32003,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,34.60
Auch,32013,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,3380.99
Augnax,32014,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,47.35
Auterive,32019,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,184.82
Ayguetinte,32024,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,78.92
Biran,32054,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,254.86
Bonas,32059,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,134.93
Castelnau-Barbarens,32076,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,313.32
Castillon-Massas,32089,CA Grand Auch Cœur de Gascogne,Gers,Occitanie,109.93


In [46]:
df.to_excel("mon_excel.xlsx", decimal=",")

TypeError: to_excel() got an unexpected keyword argument 'decimal'